<a href="https://colab.research.google.com/github/Mario2403/Unity-Python-Wrapper/blob/main/Unity_Wrapper_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#@title Instalación de Dependencias { display-mode: "form" }
#@markdown (Solo se necesitan en entornos alojados de Google COllab)

import os
from IPython.display import HTML, display

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

pro_bar = display(progress(0, 100), display_id=True)

try:
  import google.colab
  INSTALL_XVFB = True
except ImportError:
  INSTALL_XVFB = 'COLAB_ALWAYS_INSTALL_XVFB' in os.environ

if INSTALL_XVFB:
  with open('frame-buffer', 'w') as writefile:
    writefile.write("""#taken from https://gist.github.com/jterrace/2911875
XVFB=/usr/bin/Xvfb
XVFBARGS=":1 -screen 0 1024x768x24 -ac +extension GLX +render -noreset"
PIDFILE=./frame-buffer.pid
case "$1" in
  start)
    echo -n "Starting virtual X frame buffer: Xvfb"
    /sbin/start-stop-daemon --start --quiet --pidfile $PIDFILE --make-pidfile --background --exec $XVFB -- $XVFBARGS
    echo "."
    ;;
  stop)
    echo -n "Stopping virtual X frame buffer: Xvfb"
    /sbin/start-stop-daemon --stop --quiet --pidfile $PIDFILE
    rm $PIDFILE
    echo "."
    ;;
  restart)
    $0 stop
    $0 start
    ;;
  *)
        echo "Usage: /etc/init.d/xvfb {start|stop|restart}"
        exit 1
esac
exit 0
    """)
  pro_bar.update(progress(5, 100))
  !apt-get install daemon >/dev/null 2>&1
  pro_bar.update(progress(10, 100))
  !apt-get install wget >/dev/null 2>&1
  pro_bar.update(progress(20, 100))
  !wget http://security.ubuntu.com/ubuntu/pool/main/libx/libxfont/libxfont1_1.5.1-1ubuntu0.16.04.4_amd64.deb >/dev/null 2>&1
  pro_bar.update(progress(30, 100))
  !wget --output-document xvfb.deb http://security.ubuntu.com/ubuntu/pool/universe/x/xorg-server/xvfb_1.18.4-0ubuntu0.12_amd64.deb >/dev/null 2>&1
  pro_bar.update(progress(40, 100))
  !dpkg -i libxfont1_1.5.1-1ubuntu0.16.04.4_amd64.deb >/dev/null 2>&1
  pro_bar.update(progress(50, 100))
  !dpkg -i xvfb.deb >/dev/null 2>&1
  pro_bar.update(progress(70, 100))
  !rm libxfont1_1.5.1-1ubuntu0.16.04.4_amd64.deb
  pro_bar.update(progress(80, 100))
  !rm xvfb.deb
  pro_bar.update(progress(90, 100))
  !bash frame-buffer start
  os.environ["DISPLAY"] = ":1"
pro_bar.update(progress(100, 100))

try:
  import mlagents
  print("ml-agents already installed")
except ImportError:
  !python -m pip install -q mlagents==0.28.0
  print("Installed ml-agents")








Starting virtual X frame buffer: Xvfb.
     |████████████████████████████████| 164 kB 5.3 MB/s 
     |████████████████████████████████| 77 kB 5.4 MB/s 
     |████████████████████████████████| 804.1 MB 2.8 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.8.1 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.8.1 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.8.1 which is incompatible.
Installed ml-agents


# Nueva sección

In [3]:
from mlagents_envs.registry import default_registry
from mlagents_envs.base_env import ActionSpec
from mlagents_envs.environment import ActionTuple, BaseEnv

#  AC LIBRARIES
### Import libraries
import numpy as np
import gym

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.autograd import Variable


In [32]:
class UnityWrapper:

  available_envs = ['Basic', '3DBall', '3DBallHard', 'GridWorld', 'Hallway', 'VisualHallway', 'CrawlerDynamicTarget', 'CrawlerStaticTarget', 'Bouncer', 'SoccerTwos', 'PushBlock', 'VisualPushBlock', 'WallJump', 'Tennis', 'Reacher', 'Pyramids', 'VisualPyramids', 'Walker', 'FoodCollector', 'VisualFoodCollector', 'StrikersVsGoalie', 'WormStaticTarget', 'WormDynamicTarget']
  environment = None

  def __init__(self, env=None):
    self.environment_name = env
    self.set_environment(self.environment_name)
    self.init_env()

  def init_env(self):
    self.environment.reset()
    self.behavior_name = list(self.environment.behavior_specs)[0]
    self.spec = self.environment.behavior_specs[self.behavior_name]
    self.vis_obs = any(len(spec.shape) == 3 for spec in self.spec.observation_specs)

  def info(self):
    print(f"Getting info of the selected environment...")
    if (self.environment_name == "" or self.environment_name == None) or (self.environment == None) or self.behavior_name == None:
      print("[ERROR] You need to select an environment before asking for information, try using set_environment(env_name) function")
      return

    print("\n======= GENERAL INFO =======")
    print(f"The selected environment is: {self.environment_name}")
    print(f"Name of the behavior : {self.behavior_name}")
    print(f"Number of observations : {len(self.spec.observation_specs)}")
    print(f"Is there a visual observation ? {self.vis_obs}")

    print("\n======= ACTION SPACE =======")
    if self.spec.action_spec.continuous_size > 0:
      print(f"There are {self.spec.action_spec.continuous_size} continuous actions")
    if self.spec.action_spec.is_discrete():
      print(f"There are {self.spec.action_spec.discrete_size} discrete actions")

    # For discrete actions only : How many different options does each action has ?
    if self.spec.action_spec.discrete_size > 0:
      for action, branch_size in enumerate(self.spec.action_spec.discrete_branches):
        print(f"Action number {action} has {branch_size} different options")
    print("\n======= OBSERVATION INFO =======")
    self.environment.reset()

    decision_steps, terminal_steps = self.environment.get_steps(self.behavior_name)
    import matplotlib.pyplot as plt
    %matplotlib inline

    for index, obs_spec in enumerate(self.spec.observation_specs):
      if len(obs_spec.shape) == 3:
        print("Here is the first visual observation")
        plt.imshow(decision_steps.obs[index][0,:,:,:])
        plt.show()

    for index, obs_spec in enumerate(self.spec.observation_specs):
      if len(obs_spec.shape) == 1:
        print(f"First vector observations : {decision_steps.obs[index][0,:]}")
        print(f"Lenght of the observation = {len(decision_steps.obs[index][0,:])}")



  def close_env(self):
    try:
      self.environment.close()
    except:
      pass

  def set_environment(self, environment_name):
    self.close_env()
    if environment_name in self.available_envs:
      self.environment_name = environment_name
      self.environment = default_registry[self.environment_name].make()
      self.init_env()
    else:
      print(f"The selected environment {environment_name} is not in the available environment list:\n {self.available_envs}")

  def get_env(self):
    return self.environment


  def run_simulation(self):
    print("STARTING SIMULATION")
    self.environment.reset()
    for episode in range(3):
      self.environment.reset()
      decision_steps, terminal_steps = self.environment.get_steps(self.behavior_name)
      tracked_agent = -1 # -1 indicates not yet tracking
      done = False # For the tracked_agent
      episode_rewards = 0 # For the tracked_agent
      while not done:
        # Track the first agent we see if not tracking
        # Note : len(decision_steps) = [number of agents that requested a decision]
        if tracked_agent == -1 and len(decision_steps) >= 1:
          print(f"agents = {len(decision_steps)}")
          tracked_agent = decision_steps.agent_id[0]
          print(f"actual agent = {tracked_agent}")


        # Generate an action for all agents
        action = self.spec.action_spec.random_action(len(decision_steps))
        # Set the actions
        self.environment.set_actions(self.behavior_name, action)
        #print(f"action = {action.continuous}")

        # Move the simulation forward
        self.environment.step()

        # Get the new simulation results
        decision_steps, terminal_steps = self.environment.get_steps(self.behavior_name)
        if tracked_agent in decision_steps: # The agent requested a decision
          episode_rewards += decision_steps[tracked_agent].reward
        if tracked_agent in terminal_steps: # The agent terminated its episode
          episode_rewards += terminal_steps[tracked_agent].reward
          done = True
      print(f"Total rewards for episode {episode} is {episode_rewards}")
    print("END OF SIMULATION")


  def run_model(self, model):
    if model == "ActorCritic":
      self.run_AC()
    else:
      print(f"The selected model {model} is not available. You can still implement your own model. For more info visit the official man page")


  def run_AC(self, weights = None, logs_path = None, weights_path = None):

    class ActorCritic(nn.Module):
      def __init__(self, n_inputs, n_outputs):
          super(ActorCritic, self).__init__()
          self.fc1 = nn.Linear(n_inputs, 24)
          self.fc2 = nn.Linear(24, 36)
          
          self.actor = nn.Linear(36, n_outputs)
          self.critic = nn.Linear(36, 1)

      def forward(self, x):
          x = F.relu(self.fc1(x))
          x = F.relu(self.fc2(x))

          policy = self.actor(x)
          value = self.critic(x)

          return policy, value
    model = ActorCritic(self.spec.observation_specs[0].shape[0], self.spec.action_spec.discrete_branches[0]) # n_inputs , n_outputs

    algorithm = A3C(model, self.environment, weights, logs_path, weights_path)
    algorithm.train()



          



In [28]:
wrapper = UnityWrapper("Basic")

In [ ]:
wrapper.info()

Getting info of the selected environment...
======= GENERAL INFO =======
The selected environment is: Basic
Name of the behavior : Basic?team=0
Number of observations : 1
Is there a visual observation ? False
======= ACTION SPACE =======
There are 1 discrete actions
Action number 0 has 3 different options
======= OBSERVATION INFO =======
First vector observations : [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Lenght of the observation = 20


In [ ]:
wrapper.run_simulation()

In [ ]:
wrapper.run_model("ActorCritic")

In [27]:
wrapper.close_env()

In [33]:
wrapper.close_env()
wrapper = UnityWrapper("Basic")
wrapper.run_model("ActorCritic")

Se han truncado las últimas 5000 líneas del flujo de salida.
Suma de recompensas = 0.9300000108778477, media de total_values =  0.4895856146484826
=========== ACTUALIZANDO MODELO ===========
Episodio = 1527
agents = 1
Suma de recompensas = 0.9300000108778477, media de total_values =  0.4897540888418811
=========== ACTUALIZANDO MODELO ===========
Episodio = 1528
agents = 1
Suma de recompensas = 0.9300000108778477, media de total_values =  0.48992258925769216
=========== ACTUALIZANDO MODELO ===========
Episodio = 1529
agents = 1
Suma de recompensas = 0.9300000108778477, media de total_values =  0.49009120272941703
=========== ACTUALIZANDO MODELO ===========
Episodio = 1530
agents = 1
Suma de recompensas = 0.9300000108778477, media de total_values =  0.4902599803780901
=========== ACTUALIZANDO MODELO ===========
Episodio = 1531
agents = 1
Suma de recompensas = 0.9300000108778477, media de total_values =  0.4904289354903239
=========== ACTUALIZANDO MODELO ===========
Episodio = 1532
agents

KeyboardInterrupt: ignored